# Browser Automation Demo for Azure AI Foundry

This notebook demonstrates end-to-end browser automation using Azure AI Foundry Agent Service.

## What this notebook does:
1. Creates an agent that uses the Browser Automation tool to perform explorative testing against https://careers.microsoft.com/
2. Runs three independent browser sessions to capture job counts for Switzerland, Germany, and France
3. Consolidates the session output with a second agent that uses the Code Interpreter tool to generate an Excel report

## Prerequisites
- Azure AI Foundry project with configured agents
- Browser Automation connection configured in Azure AI Foundry
- Required environment variables in `.env` file

In [ ]:
# Import required libraries
from __future__ import annotations

import json
import os
import re
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, Iterable, List, Optional

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    BrowserAutomationTool,
    CodeInterpreterTool,
    ListSortOrder,
    MessageRole,
    RunStepBrowserAutomationToolCall,
    RunStepToolCallDetails,
)
from dotenv import load_dotenv

## Define Data Structures and Helper Functions

First, let's define the data structures and utility functions we'll need throughout the notebook.

In [ ]:
# Define connection ID pattern for validation
CONNECTION_ID_PATTERN = re.compile(
    r"^/subscriptions/[^/]+/resourceGroups/[^/]+/providers/[^/]+/accounts/[^/]+/projects/[^/]+/connections/[^/]+$"
)

@dataclass
class BrowserSessionResult:
    """Normalized output from a browser automation run."""
    country: str
    job_count: Optional[int]
    notes: str
    issues: List[str]
    response_text: str
    thread_id: str
    run_id: str

In [ ]:
# Environment variable helper functions
def env_or_error(name: str) -> str:
    """Read an environment variable and raise if it is missing."""
    value = os.getenv(name)
    if not value:
        raise EnvironmentError(f"Missing required environment variable: {name}")
    return value

def validate_connection_id(connection_id: str) -> str:
    """Ensure the Browser Automation connection ID matches the expected ARM path."""
    if not CONNECTION_ID_PATTERN.match(connection_id):
        missing_format = (
            "Browser Automation expects the Playwright connection ID in the form "
            "'/subscriptions/<sub>/resourceGroups/<rg>/providers/<provider>/accounts/<account>/\n"
            "projects/<project>/connections/<connection>'. Update AZURE_PLAYWRIGHT_CONNECTION_ID "
            "in your .env file with the exact value from Azure AI Foundry."
        )
        raise ValueError(missing_format)
    return connection_id

In [ ]:
# JSON parsing helper functions
def coerce_int(raw_value: Any) -> Optional[int]:
    """Convert a JSON value into an int when possible."""
    if raw_value is None:
        return None
    if isinstance(raw_value, int):
        return raw_value
    if isinstance(raw_value, float):
        return int(raw_value)
    if isinstance(raw_value, str):
        digits = re.search(r"-?\d+", raw_value)
        if digits:
            try:
                return int(digits.group(0))
            except ValueError:
                return None
    return None

def extract_json_object(text: str) -> Dict[str, Any]:
    """Extract the first JSON object embedded in text."""
    fenced_match = re.search(r"```json\s*(\{[\s\S]*?\})\s*```", text)
    raw_json = fenced_match.group(1) if fenced_match else None
    if not raw_json:
        bracket_match = re.search(r"\{[\s\S]*\}", text)
        if bracket_match:
            raw_json = bracket_match.group(0)
    if not raw_json:
        raise ValueError("Agent response did not contain JSON output.")
    return json.loads(raw_json)

## Load Environment Variables and Initialize Azure AI Client

Load the configuration from the `.env` file and set up the Azure AI Project client.

In [ ]:
# Load environment variables
load_dotenv()

# Get required configuration
project_endpoint = env_or_error("PROJECT_ENDPOINT")
model_deployment_name = env_or_error("MODEL_DEPLOYMENT_NAME")
playwright_connection_id = validate_connection_id(
    env_or_error("AZURE_PLAYWRIGHT_CONNECTION_ID")
)

print(f"✓ Project endpoint: {project_endpoint}")
print(f"✓ Model deployment: {model_deployment_name}")
print(f"✓ Playwright connection validated")

In [ ]:
# Initialize Azure AI client
credential = DefaultAzureCredential()
project_client = AIProjectClient(endpoint=project_endpoint, credential=credential)
agents_client = project_client.agents

print("✓ Azure AI Project client initialized")

## Define Browser Automation Functions

These functions handle the browser automation sessions and result processing.

In [ ]:
def summarize_browser_run_steps(agents_client, *, thread_id: str, run_id: str) -> None:
    """Print the actions performed inside a browser automation run."""
    print("  ↳ Browser automation trace:")
    run_steps = agents_client.run_steps.list(thread_id=thread_id, run_id=run_id)
    for step in run_steps:
        print(f"    Step {step.id} • status={step.status}")
        if isinstance(step.step_details, RunStepToolCallDetails):
            for call in step.step_details.tool_calls:
                if isinstance(call, RunStepBrowserAutomationToolCall):
                    details = call.browser_automation
                    print(f"      Tool call {call.id} → input: {details.input}")
                    print(f"        output: {details.output}")
                    for idx, tool_step in enumerate(details.steps or [], start=1):
                        print(f"          [{idx}] last_result={tool_step.last_step_result}")
                        print(f"              state={tool_step.current_state}")
                        if tool_step.next_step:
                            print(f"              next={tool_step.next_step}")
    print()

In [ ]:
def run_browser_session(agents_client, *, agent_id: str, country: str) -> BrowserSessionResult:
    """Execute a single browser automation session for the specified country."""
    thread = agents_client.threads.create()
    user_prompt = f"""
    Navigate to https://careers.microsoft.com/.
    Apply the location filters needed to show open job listings in {country}.
    Capture the total number of matching job postings that the site displays.
    If the filter UI fails, results are unclear, or the site blocks automation,
    document the problem.

    When you finish, respond with a single JSON object that includes the keys:
    country (string), jobCount (number or null), notes (string), issues (array of strings),
    capturedAt (ISO 8601 timestamp). Do not include additional prose or markdown.
    """
    agents_client.messages.create(
        thread_id=thread.id,
        role=MessageRole.USER,
        content=user_prompt,
    )

    run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent_id)
    print(f"Run {run.id} for {country} finished with status: {run.status}")
    if run.status == "failed":
        raise RuntimeError(f"Browser run failed for {country}: {run.last_error}")

    summarize_browser_run_steps(agents_client, thread_id=thread.id, run_id=run.id)

    response_message = agents_client.messages.get_last_message_by_role(
        thread_id=thread.id, role=MessageRole.AGENT
    )
    if not response_message or not response_message.text_messages:
        raise RuntimeError("Agent did not return a response message.")

    response_text = "\n".join(msg.text.value for msg in response_message.text_messages).strip()
    parsed = extract_json_object(response_text)
    job_count = coerce_int(parsed.get("jobCount"))
    issues = parsed.get("issues") or []
    if not isinstance(issues, list):
        issues = [str(issues)]

    notes = parsed.get("notes") or ""
    if issues:
        notes = f"{notes}\nIssues: " + "; ".join(str(item) for item in issues)

    print(f"  Result for {country}: job_count={job_count}, issues={len(issues)}")

    return BrowserSessionResult(
        country=str(parsed.get("country", country)),
        job_count=job_count,
        notes=notes.strip(),
        issues=[str(item) for item in issues],
        response_text=response_text,
        thread_id=thread.id,
        run_id=run.id,
    )

## Create Browser Automation Agent

Set up the agent that will perform the browser automation tasks.

In [ ]:
# Create browser automation tool and agent
browser_tool = BrowserAutomationTool(connection_id=playwright_connection_id)
browser_agent = agents_client.create_agent(
    model=model_deployment_name,
    name="browser-automation-tester",
    instructions=(
        "You are a QA specialist performing exploratory browser testing. Always "
        "use the Browser Automation tool when a user asks for browsing steps. "
        "After each task respond only with JSON as instructed by the user."
    ),
    tools=browser_tool.definitions,
)

print(f"✓ Created browser automation agent: {browser_agent.id}")

## Run Browser Sessions for Each Country

Execute browser automation sessions for Switzerland, Germany, and France.

In [ ]:
# Run browser sessions for each country
session_results: List[BrowserSessionResult] = []
countries = ["Switzerland", "Germany", "France"]

for country in countries:
    print(f"\n{'='*10} Running browser session for {country} {'='*10}")
    result = run_browser_session(agents_client, agent_id=browser_agent.id, country=country)
    session_results.append(result)
    print(f"✓ Completed session for {country}")

## Display Session Results

Show the collected data from all browser sessions.

In [ ]:
# Display session results summary
print("\n" + "="*50)
print("SESSION RESULTS SUMMARY")
print("="*50)

for result in session_results:
    print(f"\n{result.country}:")
    print(f"  - Job Count: {result.job_count if result.job_count is not None else 'Unknown'}")
    print(f"  - Issues: {len(result.issues)} found")
    if result.issues:
        for issue in result.issues:
            print(f"    • {issue}")
    print(f"  - Notes: {result.notes if result.notes else 'None'}")
    print(f"  - Thread ID: {result.thread_id}")
    print(f"  - Run ID: {result.run_id}")

## Generate Excel Report

Use the Code Interpreter tool to create a consolidated Excel report of the results.

In [ ]:
def generate_excel_report(
    agents_client,
    *,
    model_deployment_name: str,
    session_results: Iterable[BrowserSessionResult],
    output_dir: Path,
) -> Path:
    """Use the Code Interpreter tool to summarize results in an Excel workbook."""
    code_interpreter = CodeInterpreterTool()
    report_agent = agents_client.create_agent(
        model=model_deployment_name,
        name="browser-automation-report",
        instructions=(
            "You are an Azure AI quality analyst who turns structured findings into "
            "concise summaries. Always rely only on the provided data. Use the "
            "code interpreter tool to create artifacts when asked."
        ),
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )

    try:
        thread = agents_client.threads.create()
        payload = [
            {
                "country": result.country,
                "jobCount": result.job_count,
                "notes": result.notes,
                "issues": result.issues,
                "threadId": result.thread_id,
                "runId": result.run_id,
            }
            for result in session_results
        ]

        report_prompt = (
            "You are preparing a QA status update. Using the JSON dataset below, "
            "load it into a dataframe, normalise missing job counts to the string "
            "'Unknown', and create an Excel workbook named 'browser-testing-summary.xlsx'. "
            "Use a worksheet titled 'CareerSiteResults' that includes the columns "
            "Country, JobCount, Notes, Issues, ThreadId, RunId, ObservedAt. Set ObservedAt "
            "to today's date. After the file is saved, summarise key findings in plain text.\n\n"
            f"```json\n{json.dumps(payload, indent=2)}\n```"
        )
        agents_client.messages.create(
            thread_id=thread.id,
            role=MessageRole.USER,
            content=report_prompt,
        )

        run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=report_agent.id)
        print(f"Report run {run.id} finished with status: {run.status}")
        if run.status == "failed":
            raise RuntimeError(f"Report generation failed: {run.last_error}")

        messages = list(
            agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
        )
        report_summary = ""
        report_file_id: Optional[str] = None
        for message in messages:
            if message.role == MessageRole.AGENT:
                if message.text_messages:
                    report_summary = "\n".join(
                        text_message.text.value for text_message in message.text_messages
                    ).strip()
                if message.file_path_annotations:
                    for annotation in message.file_path_annotations:
                        report_file_id = annotation.file_path.file_id

        if not report_file_id:
            raise RuntimeError(
                "The reporting agent did not return an Excel file. Summary:\n" + report_summary
            )

        output_dir.mkdir(parents=True, exist_ok=True)
        output_path = output_dir / "browser-testing-summary.xlsx"
        agents_client.files.save(file_id=report_file_id, file_name=str(output_path))
        print(f"Saved consolidated Excel report to {output_path}")
        if report_summary:
            print("Report summary:\n" + report_summary + "\n")
        return output_path
    finally:
        agents_client.delete_agent(report_agent.id)

In [ ]:
# Generate the Excel report
output_dir = Path("reports")
report_path = generate_excel_report(
    agents_client,
    model_deployment_name=model_deployment_name,
    session_results=session_results,
    output_dir=output_dir,
)

print(f"\n✓ Excel report generated: {report_path}")

## Cleanup

Delete the agents to free up resources.

In [ ]:
# Clean up the browser agent
agents_client.delete_agent(browser_agent.id)
print(f"✓ Deleted browser automation agent: {browser_agent.id}")

# Close the project client
project_client.close()
print("✓ Closed project client connection")

## Summary

This notebook demonstrated:
1. **Browser Automation**: Automated navigation and data extraction from Microsoft Careers website
2. **Multi-country Testing**: Ran parallel sessions for different geographical locations
3. **Error Handling**: Captured and reported issues encountered during automation
4. **Report Generation**: Created an Excel report using Code Interpreter for data analysis
5. **Azure AI Integration**: Leveraged Azure AI Foundry agents with specialized tools

The generated Excel report contains detailed results from each browser session, making it easy to track job postings across different countries and identify any automation issues.